In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
path = 'C:/Users/Vladimir/Desktop/data'
objs = os.walk(path)
names = []
for d in objs:
    for f in d:
        if len(f) > 1:
            names = f

In [2]:
def isfloat(value):
    for i in range(len(value)):
        if (not value[i].isnumeric() and value[i] != ',' and value[i] != '.'):
            return False
    return True
data = []
parameter = []
Country_name = ""
Country_information = []
cntrs = []
for name in names:
    file = open(path+'/'+name, 'r')
    st = [1,2]
    parameter.append(name[:len(name)-4])
    while (len(st) > 1):
        st = file.readline().split()
        if (len(st) > 1):
            i = 0;
            while (st[i] != 'None' and not isfloat(st[i])):
                Country_name += st[i]+' '
                i += 1
            Country_name = Country_name[:-1]
            Country_information.append(Country_name)
            while (i < len(st)):
                if (st[i]  == 'None'):
                    Country_information.append(st[i])
                else:
                    st[i] = st[i].replace(',','')
                    Country_information.append(float(st[i]))
                i += 1
            i = 0
            cntrs.append(np.array(Country_information))
        Country_information.clear()
        Country_name = ""
    parameter.append(np.array(cntrs))
    cntrs = []
    data.append(parameter)
    parameter = []
data.pop()
print(len(data))

13


In [3]:
for i in range(len(names)):
    names[i] = names[i][:-4]
print(names)

['Burglary and housebraking rate', 'Burglary and housebraking', 'Burglary rate', 'Burglary', 'Homicide rate', 'Homicide', 'Kidnapping rate', 'Kidnapping', 'Private cars theft rate', 'Private cars theft', 'Rape rate', 'Rape', 'Robbery rate', 'Robbery']


In [4]:
coeffs = { names[0] : 0.25, names[2] : 0.25, names[4] : 12, names[6] : 15, names[8] : 1, names[10] : 10, names[12] : 7}
print(coeffs)

{'Burglary and housebraking rate': 0.25, 'Burglary rate': 0.25, 'Homicide rate': 12, 'Kidnapping rate': 15, 'Private cars theft rate': 1, 'Rape rate': 10, 'Robbery rate': 7}


In [5]:
def av(arr):
    result = 0
    err_cnt = 0
    for elem in arr:
        if elem == 'None':
            err_cnt += 1
        else:
            result += float(elem)
    if (err_cnt > 5):
        return -1
    else:
        result /= (len(arr) - err_cnt)
        return result
def ac(arr):
    n = len(arr)
    if (arr[n-1] == 'None' or arr[n-2] == 'None' or arr[n-3] == 'None'):
        return 0
    a = float(arr[n-1])
    b = float(arr[n-2])
    c = float(arr[n-3])
    eps = (a + b + c)/100
    if (a < b < c):
        return 1
    elif (a > b - eps > c - eps):
        return -1
    else:
        return 0

In [6]:
rating = {}
velocity = {}
for spec in data:
    if (spec[0].find('rate') != -1):
        for country in spec[1]:

            if (velocity.get(country[0]) == None):
                velocity[country[0]] = ac(country[1:])
            else:
                velocity[country[0]] += ac(country[1:])

            if (rating.get(country[0]) == None):
                val = av(country[1:])
                if (val != -1):
                    rating[country[0]] = val * coeffs[spec[0]]
                else:
                    if (coeffs[spec[0]] <= 7):
                        rating[country[0]] = 0
                    else:
                        rating[country[0]] = -1
            elif (rating[country[0]] != -1):
                val = av(country[1:])
                if (val != -1):
                    rating[country[0]] += val * coeffs[spec[0]]
                else:
                    rating[country[0]] = -1
print(velocity['United States of America'])

4


In [7]:
import geojson
file = open('map.geojson', 'r+')
mapping = geojson.load(file)

In [8]:
for a in mapping['features']:
    abr = a['properties']['adm0_a3']
    name = a['properties']['name_long']
    a['properties'].clear()
    a['properties']['Abbreviation'] = abr
    a['properties']['Name'] = name
    if (rating.get(name) != None):
        a['properties']['Crime rating'] = int(rating[name])
        if (rating[name] < 0):
            a['properties']['Qonqlusion'] = 'None'
        elif (rating[name] < 500):
            a['properties']['Qonqlusion'] = 'Very good'
        elif (rating[name] < 900):
            a['properties']['Qonqlusion'] = 'Good'
        elif (rating[name] < 1500):
            a['properties']['Qonqlusion'] = 'Normal'
        elif (rating[name] < 2000):
            a['properties']['Qonqlusion'] = 'Increased'
        elif (rating[name] < 3000):
            a['properties']['Qonqlusion'] = 'High'
        elif (rating[name] < 4000):
            a['properties']['Qonqlusion'] = 'Very high'
        else:
            a['properties']['Qonqlusion'] = 'Deadly'
        if (velocity[name] > 1):
            a['properties']['Dynamic'] = "Improving " + "(+" + str(velocity[name]) + ")"
        elif (velocity[name] >= -1):
            a['properties']['Dynamic'] = "Stable"
        else:
            a['properties']['Dynamic'] = "Worsening " + "(" + str(velocity[name]) + ")"
    else:
        a['properties']['Crime rating'] = -1
        a['properties']['Qonqlusion'] = 'None'
        a['properties']['Dynamic'] = 0

In [9]:
###Output
